In [3]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

# 获取第一帧
ret, frame1 = cap.read()

# 转换为灰度图像
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# 创建一个蒙版用于绘制轨迹
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

while (1):
    ret, frame2 = cap.read()
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # 计算光流
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # 绘制光流轨迹
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)



    cv2.imshow('frame', bgr)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # 更新帧和蒙版
    prvs = next

cap.release()
cv2.destroyAllWindows()

In [3]:
import numpy as np
import cv2
# import sklearn
# from sklearn.metrics.pairwise import cosine_similarity

step = 10

if __name__ == '__main__':
    cam = cv2.VideoCapture(0)
    ret, prev = cam.read()
    prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

    while True:
        ret, img = cam.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # 使用Gunnar Farneback算法计算密集光流
        # flow=cv2.calcOpticalFlowPyrLK(prevgray,gray,)
        flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        prevgray = gray

        # 绘制线
        h, w = gray.shape[:2]
        y, x = np.mgrid[step / 2:h:step, step / 2:w:step].reshape(2, -1).astype(int)
        fx, fy = flow[y, x].T
        lines = np.vstack([x, y, x + fx, y + fy]).T.reshape(-1, 2, 2)
        lines = np.int32(lines)

        line = []
        for l in lines:
            # if l[0][0] - l[1][0] > 3 or l[0][1] - l[1][1] > 3:
            line.append(l)

        # flow_avg = np.zeros(2)
        # flow_avg[0] = np.mean(flow[..., 0])
        # flow_avg[1] = np.mean(flow[..., 1])
        #
        # cos_sim = 0
        # for i in range(flow.shape[0]):
        #     for j in range(flow.shape[1]):
        #         vec1 = flow[i, j]
        #         vec2 = flow_avg
        #         b = (np.linalg.norm(vec1) * np.linalg.norm(vec2))
        #         if b == 0:
        #             cos_sim+=-1
        #             continue
        #         cos_sim += vec1.dot(vec2) / b
        # cos_sim /= flow.shape[0] + flow.shape[1]

        flow_avg = np.zeros(2)
        flow_avg[0] = np.mean(flow[..., 0])
        flow_avg[1] = np.mean(flow[..., 1])
        flow_len=np.linalg.norm(flow_avg)
        if flow_len>5:
            print('画面被移动了',flow_len)

        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(flow_len), (10, 100), font, 0.5, (255, 255, 0), 2)
        # if flow_len > 5:
        #     print('摄像头被移动')


        cv2.polylines(img, line, 0, (0, 255, 255))
        cv2.imshow('flow', img)

        ch = cv2.waitKey(5)
        if ch == 27:
            break
    cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

# 设置ShiTomasi角点检测的参数
feature_params = dict(maxCorners=100,
                      qualityLevel=0.3,
                      minDistance=7,
                      blockSize=7)

# 设置Lucas-Kanade光流法的参数
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# 创建随机颜色
color = np.random.randint(0, 255, (100, 3))

# 获取第一帧，找到角点
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# 创建一个掩膜用于绘制轨迹
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 计算光流
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # 选择好的点
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    flow=good_new-good_old
    flow_avg = np.zeros(2)
    flow_avg[0] = np.mean(flow[...,0])
    flow_avg[1] = np.mean(flow[..., 1])
    flow_len = np.linalg.norm(flow_avg)
    print(flow_len)


    # 绘制轨迹
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        a,b,c,d=int(a),int(b),int(c),int(d)
        mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2)
        frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)
    img = cv2.add(frame, mask)

    cv2.imshow('frame', img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # 更新上一帧的图像和角点位置
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cv2.destroyAllWindows()
cap.release()

0.010160340686571321
0.008998367259845451
0.029601959348461236
0.016753374260486217
0.006655686070368333
0.010883508722386942
0.016901380445669256
0.026071039826070848
0.03526426290471844
0.0207188175367939
0.014623882690749605
0.019541538267292255
0.019073015662572877
0.01592170848583413
0.024624631870839363
0.015971488235785335
0.01991247802836556
3.288532113288056e-08
0.012502339788157188
0.008966921574451208
0.012373771472227341
0.01120069404713866
0.028047646277615787
0.010565432159187649
0.01108244993844994
0.0
0.019902786898450278
0.028902256752981872
0.01611268358012643
0.04184435347723038
0.0026317691105612047
0.009182841512776818
0.010135764350425135
0.0
0.010613309053245817
0.01694079904857112
0.011136379446305918
0.0323373842910437
0.01973127038686037
0.013736004178393267
0.007855864246228838
0.0
0.0208360257307404
0.011865508898506466
0.012524568975923295
0.01591757638415051
0.03159501244761801
0.011865192192525457
0.015799756949809333
3.288532113288056e-08
0.0500509098656

C:\ProgramData\Miniconda3\envs\General\lib\site-packages\numpy\core\fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Miniconda3\envs\General\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


TypeError: 'NoneType' object is not subscriptable